## Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods (Week 1)

## Introduction/Business Problem

Many tourists look for fast and sometimes cheaper options for food on their trips.
Being in unknown cities brings the challenge of finding the best options and technology can be a great ally.
In this experiment, let's consider a tourist visiting the U.S. and looking for the highest concentration of pizza places in some cities.

# Data
We will use Foursquare to gather information about the most concentrated pizza places in five major cities in the US:
Chicago, NY, San Francisco, Jersey and Boston

In [2]:
#Import Libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize
import folium

In [9]:
#Foursquare setup
CLIENT_ID = 'VJHLADU2203W12PK0M43XNUAN3RBWLBOR0FDQF3HBQDG4PHJ'
CLIENT_SECRET = '' #Hidding client secret
VERSION = '20180605'
LIMIT = 500
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Jersey City, NJ', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1ca941735")
    results[city] = requests.get(url).json()

In [11]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [25]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Pizza places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Pizza places in New York, NY =  306
Showing Top 100
Pizza places in Chicago, IL =  230
Showing Top 100
Pizza places in San Francisco, CA =  167
Showing Top 100
Pizza places in Jersey City, NJ =  123
Showing Top 100
Pizza places in Boston, MA =  186
Showing Top 100


In [13]:
maps[cities[0]]

In [14]:
maps[cities[1]]

In [15]:
maps[cities[2]]

In [16]:
maps[cities[3]]

In [17]:
maps[cities[4]]